In [ ]:
# -*- coding: utf-8 -*-
"""
R&R 프로젝트의 최종 데이터 세트를 완성하는 스크립트입니다.

[작업 순서]
1. 보강 완료된 영화 데이터와 연관된 모든 DB용 파일 로드
2. 영화 데이터 중 필수 정보(감독, 배급사, 커버 이미지)가 없는 결측치 식별
3. 해당 영화(work_id)들을 모든 관련 데이터프레임에서 일괄적으로 제거
4. 최종적으로 정리된(cleaned) 데이터 파일들 저장
"""

import pandas as pd

# ==============================================================================
# Step 1: 모든 관련 데이터 파일 로드
# ==============================================================================
print("Step 1: 보강 완료된 데이터와 연관 파일들을 로드합니다.")

try:
    # 이전에 보강 완료된 최종 영화 데이터
    movies_final_df = pd.read_csv("db_movies_final.csv")

    # 영화 데이터와 관계를 맺고 있는 다른 테이블들
    works_df = pd.read_csv("db_works.csv")
    work_genre_df = pd.read_csv("db_work_genre.csv")
    ratings_df = pd.read_csv("db_ratings.csv")

    print("모든 데이터 파일 로드 완료.")

except FileNotFoundError as e:
    print(f"❌ 오류: 필수 파일이 없습니다 - {e.filename}")
    print("이전 단계에서 생성된 db_movies_final.csv, db_works.csv, db_work_genre.csv, db_ratings.csv 파일이 모두 있는지 확인해주세요.")
    exit()

# ==============================================================================
# Step 2: 결측치 데이터 식별
# ==============================================================================
print("\nStep 2: 필수 정보가 누락된 영화 데이터를 식별합니다.")

# 감독(director), 배급사(publisher) 또는 커버 이미지(cover_img) 정보가 없는 영화를 결측치로 판단
# .fillna('')는 혹시 모를 NaN 값을 빈 문자열로 바꿔줍니다.
missing_movies = movies_final_df[
    (movies_final_df['director'].fillna('').str.strip() == '') |
    (movies_final_df['publisher'].fillna('').str.strip() == '') |
    (movies_final_df['cover_img'].fillna('').str.strip() == '')
]

# 제거할 영화들의 work_id 목록 생성
ids_to_remove = missing_movies['work_id'].tolist()

if ids_to_remove:
    print(f"총 {len(ids_to_remove)}개의 영화에서 결측치가 발견되어 제거를 진행합니다.")
else:
    print("결측치가 발견되지 않았습니다. 모든 영화 데이터가 유효합니다.")

# ==============================================================================
# Step 3: 모든 관련 데이터프레임에서 결측치 일괄 제거
# ==============================================================================
if ids_to_remove:
    print("\nStep 3: 모든 관련 테이블에서 해당 영화 데이터를 일괄 제거합니다.")

    # isin() 함수를 사용하여 제거할 ID 목록에 포함되지 않은 데이터만 남김
    movies_cleaned_df = movies_final_df[~movies_final_df['work_id'].isin(ids_to_remove)]
    works_cleaned_df = works_df[~works_df['work_id'].isin(ids_to_remove)]
    work_genre_cleaned_df = work_genre_df[~work_genre_df['work_id'].isin(ids_to_remove)]
    ratings_cleaned_df = ratings_df[~ratings_df['work_id'].isin(ids_to_remove)]

    print(f"Movies 테이블 정리 완료: {len(movies_final_df)} -> {len(movies_cleaned_df)}")
    print(f"Works 테이블 정리 완료: {len(works_df)} -> {len(works_cleaned_df)}")
    print(f"Work_Genre 테이블 정리 완료: {len(work_genre_df)} -> {len(work_genre_cleaned_df)}")
    print(f"Ratings 테이블 정리 완료: {len(ratings_df)} -> {len(ratings_cleaned_df)}")
else:
    # 제거할 데이터가 없으면 기존 데이터프레임을 그대로 사용
    movies_cleaned_df = movies_final_df
    works_cleaned_df = works_df
    work_genre_cleaned_df = work_genre_df
    ratings_cleaned_df = ratings_df

# ==============================================================================
# Step 4: 최종 정리된 파일 저장
# ==============================================================================
print("\nStep 4: 최종적으로 정리된 데이터 파일들을 저장합니다.")

# 최종 파일 이름에 '_cleaned' 접미사 추가
movies_cleaned_df.to_csv("db_movies_cleaned.csv", index=False, encoding='utf-8-sig')
works_cleaned_df.to_csv("db_works_cleaned.csv", index=False, encoding='utf-8-sig')
work_genre_cleaned_df.to_csv("db_work_genre_cleaned.csv", index=False, encoding='utf-8-sig')
ratings_cleaned_df.to_csv("db_ratings_cleaned.csv", index=False, encoding='utf-8-sig')

print("\n🎉 모든 데이터의 최종 정리가 완료되었습니다!")
print("이제 '_cleaned'가 붙은 파일들을 동료에게 전달하여 DB 시딩을 진행하세요.")



Step 1: 보강 완료된 데이터와 연관 파일들을 로드합니다.
모든 데이터 파일 로드 완료.

Step 2: 필수 정보가 누락된 영화 데이터를 식별합니다.
총 462개의 영화에서 결측치가 발견되어 제거를 진행합니다.

Step 3: 모든 관련 테이블에서 해당 영화 데이터를 일괄 제거합니다.
Movies 테이블 정리 완료: 9708 -> 9246
Works 테이블 정리 완료: 29800 -> 29338
Work_Genre 테이블 정리 완료: 51079 -> 50159
Ratings 테이블 정리 완료: 100836 -> 99598

Step 4: 최종적으로 정리된 데이터 파일들을 저장합니다.

🎉 모든 데이터의 최종 정리가 완료되었습니다!
이제 '_cleaned'가 붙은 파일들을 동료에게 전달하여 DB 시딩을 진행하세요.


In [ ]:
movies_cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9246 entries, 0 to 9707
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   work_id     9246 non-null   int64  
 1   name        9246 non-null   object 
 2   created_at  9246 non-null   object 
 3   director    9246 non-null   object 
 4   publisher   9246 non-null   object 
 5   ai_summary  0 non-null      float64
 6   cover_img   9246 non-null   object 
 7   reward      0 non-null      float64
dtypes: float64(2), int64(1), object(5)
memory usage: 650.1+ KB


In [ ]:
ratings_cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99598 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   99598 non-null  int64  
 1   work_id  99598 non-null  int64  
 2   rating   99598 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 3.0 MB


In [ ]:
work_genre_cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50159 entries, 0 to 51078
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   work_id   50159 non-null  int64
 1   genre_id  50159 non-null  int64
dtypes: int64(2)
memory usage: 1.1 MB


In [ ]:
works_cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29338 entries, 0 to 29799
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   work_id  29338 non-null  int64  
 1   type     29338 non-null  object 
 2   rating   29338 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 916.8+ KB


In [ ]:
books_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20092 entries, 0 to 20091
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   work_id     20092 non-null  int64  
 1   name        20092 non-null  object 
 2   author      19336 non-null  object 
 3   ISBN        11083 non-null  object 
 4   publisher   0 non-null      float64
 5   created_at  20092 non-null  object 
 6   ai_summary  0 non-null      float64
 7   cover_img   4584 non-null   object 
 8   reward      0 non-null      float64
dtypes: float64(3), int64(1), object(5)
memory usage: 1.4+ MB


In [ ]:
# -*- coding: utf-8 -*-
"""
R&R 프로젝트 - Books 데이터 최종 정리 스크립트

1. 보강 완료된 책 데이터와 연관된 모든 DB용 파일 로드
2. 필수 정보(author, publisher, cover_img) 결측치 확인
3. 해당 책(work_id)들을 모든 관련 테이블에서 일괄 제거
4. 최종적으로 정리된(cleaned) 데이터 파일 저장
"""

import pandas as pd

# ==============================================================================
# Step 1: 모든 관련 데이터 파일 로드
# ==============================================================================
print("Step 1: 보강 완료된 책 데이터와 연관 파일들을 로드합니다.")

try:
    books_final_df = pd.read_csv("db_books_final.csv")
    works_df = pd.read_csv("db_works.csv")
    work_genre_df = pd.read_csv("db_work_genre.csv")
    ratings_df = pd.read_csv("db_ratings.csv")
    print("모든 데이터 파일 로드 완료.")

except FileNotFoundError as e:
    print(f"❌ 오류: 필수 파일이 없습니다 - {e.filename}")
    print("db_books_final.csv, db_works.csv, db_work_genre.csv, db_ratings.csv 파일이 모두 있는지 확인해주세요.")
    exit()

# ==============================================================================
# Step 2: 결측치 데이터 식별
# ==============================================================================
print("\nStep 2: 필수 정보가 누락된 책 데이터를 식별합니다.")

missing_books = books_final_df[
    (books_final_df['author'].fillna('').str.strip() == '')
]

ids_to_remove = missing_books['work_id'].tolist()

if ids_to_remove:
    print(f"총 {len(ids_to_remove)}개의 책에서 결측치가 발견되어 제거를 진행합니다.")
else:
    print("결측치가 발견되지 않았습니다. 모든 책 데이터가 유효합니다.")

# ==============================================================================
# Step 3: 모든 관련 테이블에서 결측치 일괄 제거
# ==============================================================================
if ids_to_remove:
    print("\nStep 3: 모든 관련 테이블에서 해당 책 데이터를 일괄 제거합니다.")

    books_cleaned_df = books_final_df[~books_final_df['work_id'].isin(ids_to_remove)]
    works_cleaned_df = works_df[~works_df['work_id'].isin(ids_to_remove)]
    work_genre_cleaned_df = work_genre_df[~work_genre_df['work_id'].isin(ids_to_remove)]
    ratings_cleaned_df = ratings_df[~ratings_df['work_id'].isin(ids_to_remove)]

    print(f"Books 테이블 정리 완료: {len(books_final_df)} -> {len(books_cleaned_df)}")
    print(f"Works 테이블 정리 완료: {len(works_df)} -> {len(works_cleaned_df)}")
    print(f"Work_Genre 테이블 정리 완료: {len(work_genre_df)} -> {len(work_genre_cleaned_df)}")
    print(f"Ratings 테이블 정리 완료: {len(ratings_df)} -> {len(ratings_cleaned_df)}")
else:
    books_cleaned_df = books_final_df
    works_cleaned_df = works_df
    work_genre_cleaned_df = work_genre_df
    ratings_cleaned_df = ratings_df

# ==============================================================================
# Step 4: 최종 정리된 파일 저장
# ==============================================================================
print("\nStep 4: 최종적으로 정리된 데이터 파일들을 저장합니다.")

books_cleaned_df.to_csv("db_books_cleaned.csv", index=False, encoding='utf-8-sig')
works_cleaned_df.to_csv("db_works_cleaned.csv", index=False, encoding='utf-8-sig')
work_genre_cleaned_df.to_csv("db_work_genre_cleaned.csv", index=False, encoding='utf-8-sig')
ratings_cleaned_df.to_csv("db_ratings_cleaned.csv", index=False, encoding='utf-8-sig')

print("\n🎉 모든 Books 관련 데이터 최종 정리 완료!")
print("이제 '_cleaned'가 붙은 파일들을 DB 시딩에 사용하세요.")


Step 1: 보강 완료된 책 데이터와 연관 파일들을 로드합니다.
모든 데이터 파일 로드 완료.

Step 2: 필수 정보가 누락된 책 데이터를 식별합니다.
총 756개의 책에서 결측치가 발견되어 제거를 진행합니다.

Step 3: 모든 관련 테이블에서 해당 책 데이터를 일괄 제거합니다.
Books 테이블 정리 완료: 20092 -> 19336
Works 테이블 정리 완료: 29338 -> 28582
Work_Genre 테이블 정리 완료: 50159 -> 49214
Ratings 테이블 정리 완료: 99598 -> 99598

Step 4: 최종적으로 정리된 데이터 파일들을 저장합니다.

🎉 모든 Books 관련 데이터 최종 정리 완료!
이제 '_cleaned'가 붙은 파일들을 DB 시딩에 사용하세요.


In [ ]:
books_cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19336 entries, 0 to 20091
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   work_id     19336 non-null  int64  
 1   name        19336 non-null  object 
 2   author      19336 non-null  object 
 3   ISBN        11083 non-null  object 
 4   publisher   0 non-null      float64
 5   created_at  19336 non-null  object 
 6   ai_summary  0 non-null      float64
 7   cover_img   4584 non-null   object 
 8   reward      0 non-null      float64
dtypes: float64(3), int64(1), object(5)
memory usage: 1.5+ MB


In [ ]:
import pandas as pd

# ======================================================================
# Step 0: cleaned 데이터 로드
# ======================================================================
print("✅ cleaned 데이터 파일들을 로드합니다.")

movies_cleaned = pd.read_csv("db_movies_cleaned.csv")
books_cleaned = pd.read_csv("db_books_cleaned.csv")
works_cleaned = pd.read_csv("db_works_cleaned.csv")
work_genre_cleaned = pd.read_csv("db_work_genre_cleaned.csv")
ratings_cleaned = pd.read_csv("db_ratings_cleaned.csv")

# ======================================================================
# Step 1: work_id 무결성 체크
# ======================================================================
print("\n🔍 Step 1: work_id 무결성 확인")

# 모든 테이블에서 work_id 추출
movie_ids = set(movies_cleaned['work_id'].unique())
book_ids = set(books_cleaned['work_id'].unique())
works_ids = set(works_cleaned['work_id'].unique())
genre_ids = set(work_genre_cleaned['work_id'].unique())
rating_ids = set(ratings_cleaned['work_id'].unique())

print(f"Movies count: {len(movie_ids)}")
print(f"Books count: {len(book_ids)}")
print(f"Works count: {len(works_ids)}")
print(f"Work_Genre count: {len(genre_ids)}")
print(f"Ratings count: {len(rating_ids)}")

# ======================================================================
# Step 2: 교집합과 차집합 확인
# ======================================================================
print("\n🔍 Step 2: 테이블 간 일관성 검증")

# 1) works에 없는 ID가 다른 테이블에 있는지?
for table_name, ids in {
    "Movies": movie_ids,
    "Books": book_ids,
    "Work_Genre": genre_ids,
    "Ratings": rating_ids,
}.items():
    diff = ids - works_ids
    if diff:
        print(f"⚠️ {table_name}에만 존재하고 Works에는 없는 ID {len(diff)}개 발견")
    else:
        print(f"✅ {table_name} ↔ Works 일치")

# 2) works에는 있는데 movies/books 양쪽에 없는 work_id가 있는지?
works_only = works_ids - (movie_ids | book_ids)
if works_only:
    print(f"⚠️ Works에는 있으나 Movies/Books에 없는 work_id {len(works_only)}개 발견")
else:
    print("✅ Works ↔ Movies/Books 매핑 정상")

# ======================================================================
# Step 3: 삭제된 work_id 검증
# ======================================================================
print("\n🔍 Step 3: 삭제된 work_id가 정말 전부 제거됐는지 확인")

# 원본 파일 불러오기
movies_final = pd.read_csv("db_movies_final.csv")
books_final = pd.read_csv("db_books_final.csv")
works_final = pd.read_csv("db_works.csv")
work_genre_final = pd.read_csv("db_work_genre.csv")
ratings_final = pd.read_csv("db_ratings.csv")

# 원본 대비 cleaned 비교
removed_from_movies = set(movies_final['work_id']) - set(movies_cleaned['work_id'])
removed_from_books = set(books_final['work_id']) - set(books_cleaned['work_id'])
removed_total = removed_from_movies | removed_from_books

print(f"총 제거된 work_id 수: {len(removed_total)}")

# 다른 테이블에서도 모두 제거됐는지 확인
for table_name, df in {
    "Works": works_cleaned,
    "Work_Genre": work_genre_cleaned,
    "Ratings": ratings_cleaned,
}.items():
    leftover = removed_total & set(df['work_id'].unique())
    if leftover:
        print(f"❌ {table_name}에서 {len(leftover)}개 work_id가 아직 남아있음")
    else:
        print(f"✅ {table_name}에서도 전부 제거 완료")


✅ cleaned 데이터 파일들을 로드합니다.

🔍 Step 1: work_id 무결성 확인
Movies count: 9246
Books count: 19336
Works count: 28582
Work_Genre count: 28582
Ratings count: 9228

🔍 Step 2: 테이블 간 일관성 검증
✅ Movies ↔ Works 일치
✅ Books ↔ Works 일치
✅ Work_Genre ↔ Works 일치
✅ Ratings ↔ Works 일치
✅ Works ↔ Movies/Books 매핑 정상

🔍 Step 3: 삭제된 work_id가 정말 전부 제거됐는지 확인
총 제거된 work_id 수: 1218
✅ Works에서도 전부 제거 완료
✅ Work_Genre에서도 전부 제거 완료
✅ Ratings에서도 전부 제거 완료


In [ ]:
import pandas as pd

# ✅ cleaned 데이터 로드
works_df = pd.read_csv("db_works_cleaned.csv")
ratings_df = pd.read_csv("db_ratings_cleaned.csv")

# 🔍 Step 1: Ratings에 존재하는 work_id가 Works에도 존재하는지 확인
invalid_ids = set(ratings_df['work_id']) - set(works_df['work_id'])

if len(invalid_ids) == 0:
    print("✅ Ratings 무결성 검증 통과: 모든 work_id가 Works에 존재합니다.")
else:
    print(f"⚠️ Ratings에만 존재하는 work_id 발견: {len(invalid_ids)}개")
    print("예시:", list(invalid_ids)[:10])

# 🔍 Step 2: Works 대비 Ratings 커버리지 확인
print(f"Works count: {len(works_df)}")
print(f"Ratings count: {len(ratings_df)}")
print(f"Ratings에서 참조된 고유 work_id 수: {ratings_df['work_id'].nunique()}")


⚠️ Ratings에만 존재하는 work_id 발견: 34개
예시: [166024, 122888, 122896, 167570, 114335, 159779, 181413, 141866, 172591, 143410]
Works count: 28582
Ratings count: 99598
Ratings에서 참조된 고유 work_id 수: 9262


In [ ]:
# -*- coding: utf-8 -*-
"""
Ratings 정리 스크립트
- Works에 존재하지 않는 work_id가 Ratings에 있으면 제거 (orphan 제거)
- cleaned 파일이 있으면 우선 사용, 없으면 원본 사용
- 결과를 db_ratings_cleaned.csv 로 저장하고 무결성 재확인
"""

import os
import pandas as pd

def load_csv_any(name: str) -> pd.DataFrame:
    """
    db_{name}_cleaned.csv 가 있으면 우선 사용, 없으면 db_{name}.csv 사용
    """
    cleaned = f"db_{name}_cleaned.csv"
    raw = f"db_{name}.csv"
    if os.path.exists(cleaned):
        print(f"load: {cleaned}")
        return pd.read_csv(cleaned)
    elif os.path.exists(raw):
        print(f"load: {raw}")
        return pd.read_csv(raw)
    else:
        raise FileNotFoundError(f"둘 다 없음: {cleaned}, {raw}")

# ----------------------------------------------------------------------
# 1) 데이터 로드
# ----------------------------------------------------------------------
print("✅ 데이터 로드")
ratings = load_csv_any("ratings")
works   = load_csv_any("works")

# 타입 안전장치(필요 시)
for df, nm in [(ratings, "ratings"), (works, "works")]:
    if df["work_id"].dtype != "int64" and df["work_id"].dtype != "int32":
        # CSV에서 실수로 읽힌 경우를 대비해 정수로 캐스팅
        df["work_id"] = pd.to_numeric(df["work_id"], errors="coerce").astype("Int64")

# ----------------------------------------------------------------------
# 2) orphan 제거: Works에 없는 work_id를 Ratings에서 삭제
# ----------------------------------------------------------------------
print("\n🔧 orphan 제거 수행")
before = len(ratings)
works_ids = set(works["work_id"].dropna().astype(int).unique())
ratings["work_id"] = ratings["work_id"].astype("Int64")  # 일관성
ratings_cleaned = ratings[ratings["work_id"].isin(works_ids)].copy()
after = len(ratings_cleaned)
removed = before - after
print(f"Ratings: {before} -> {after} (제거 {removed}건)")

# (선택) 중복 제거가 필요하면 주석 해제
# 키가 (user_id, work_id) 라고 가정하고 가장 최근/마지막 레코드만 유지
# if {"user_id", "work_id"}.issubset(ratings_cleaned.columns):
#     d_before = len(ratings_cleaned)
#     ratings_cleaned = ratings_cleaned.sort_index().drop_duplicates(subset=["user_id","work_id"], keep="last")
#     print(f"중복 제거: {d_before} -> {len(ratings_cleaned)}")

# ----------------------------------------------------------------------
# 3) 저장
# ----------------------------------------------------------------------
out_path = "db_ratings_cleaned.csv"
ratings_cleaned.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"\n💾 저장 완료: {out_path}")

# ----------------------------------------------------------------------
# 4) 무결성 재검증 (Works 기준)
# ----------------------------------------------------------------------
print("\n🔍 무결성 재검증")
rating_ids = set(ratings_cleaned["work_id"].astype(int).unique())
diff = rating_ids - works_ids
if diff:
    print(f"⚠️ 여전히 Works에 없는 work_id {len(diff)}건 존재")
else:
    print("✅ Ratings ↔ Works 일치 (orphan 없음)")


✅ 데이터 로드
load: db_ratings_cleaned.csv
load: db_works_cleaned.csv

🔧 orphan 제거 수행
Ratings: 99598 -> 99551 (제거 47건)

💾 저장 완료: db_ratings_cleaned.csv

🔍 무결성 재검증
✅ Ratings ↔ Works 일치 (orphan 없음)
